In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_numerics = pd.read_csv('./data/numerics.csv')
df_signal = pd.read_csv('./data/signals.csv')
df_breaths = pd.read_csv('./data/breaths.csv')

In [4]:
# merge signals and numerics. The frequency will be 125 Hz. 
df_signal_numeric = pd.merge(df_signal, df_numerics, how='left', left_on=['user_id', 'time'], right_on=['user_id', 'Time [s]'])

In [8]:

df_signal_numeric['ann1_is_breath'] = np.nan
df_signal_numeric['ann2_is_breath'] = np.nan

In [ ]:
#

for index, row in df_breaths.iterrows():
    signal_patient_index_ann1 = row['breaths ann1 [signal sample no]']
    signal_patient_index_ann2 = row[' breaths ann2 [signal sample no]']
    userid = row['user_id']
    try:
        #find the exact indiex where a breath was recorded by 1st manual annotator
        ann1_is_breath_index = df_signal_numeric.loc[(df_signal_numeric['signal_patient_index'] == row['breaths ann1 [signal sample no]']) & 
                                  (df_signal_numeric['user_id'] == row['user_id'])].index.values[0]

        df_signal_numeric.at[ann1_is_breath_index, 'ann1_is_breath'] = 1
    except Exception as e:
        print(e)
        pass
    
    try:
        #find the exact indiex where a breath was recorded by 2nd manual annotator
        ann2_is_breath_index = df_signal_numeric.loc[(df_signal_numeric['signal_patient_index'] == row[' breaths ann2 [signal sample no]']) & 
                              (df_signal_numeric['user_id'] == row['user_id'])].index.values[0]

        df_signal_numeric.at[ann2_is_breath_index, 'ann2_is_breath'] = 1
    except Exception as e:
        print(e)
        pass


In [ ]:
df_signal_numeric['ann1_is_breath'].value_counts()

In [ ]:
df_signal_numeric['ann2_is_breath'].value_counts()

In [ ]:
# save a checkpoint for the data
df_signal_numeric.to_csv('./data/merged_df.csv', index = False)

In [9]:
# df_signal_numeric = pd.read_csv('./data/merged_df.csv')

In [11]:
df_signal_numeric['ann1_is_breath'] = df_signal_numeric['ann1_is_breath'].fillna(value=0)
df_signal_numeric['ann2_is_breath'] = df_signal_numeric['ann2_is_breath'].fillna(value=0)

In [12]:
df_signal_numeric['ann1_breathspm'] = np.nan
df_signal_numeric['ann2_breathspm'] = np.nan

In [13]:
df_breaths['ann1_breathspm'] = np.nan
df_breaths['ann2_breathspm'] = np.nan

In [32]:
df_breaths['ann1_berathspm'].max()

inf

In [15]:
# calculate the breaths per minute

df_breaths['ann1_berathspm'] = 60*125/(df_breaths['breaths ann1 [signal sample no]'] - df_breaths['ann1_prev_breath'])
df_breaths['ann2_berathspm'] = 60*125/(df_breaths[' breaths ann2 [signal sample no]'] - df_breaths['ann2_prev_breath'])

In [16]:
df_breaths['ann1_berathspm'] = df_breaths['ann1_berathspm'].apply(lambda x : "{0:.0f}".format(x)).replace('nan', np.nan)
df_breaths['ann2_berathspm'] = df_breaths['ann2_berathspm'].apply(lambda x : "{0:.0f}".format(x)).replace('nan', np.nan)

In [17]:
df_breaths['ann1_berathspm']= pd.to_numeric(df_breaths['ann1_berathspm'], downcast='integer')
df_breaths['ann2_berathspm']= pd.to_numeric(df_breaths['ann2_berathspm'], downcast='integer')

In [18]:
df_breaths.dtypes

breaths ann1 [signal sample no]     float64
 breaths ann2 [signal sample no]    float64
user_id                               int64
delta_ann1                          float64
delta_ann2                          float64
ann1_prev_breath                    float64
ann2_prev_breath                    float64
ann1_breathspm                      float64
ann2_breathspm                      float64
ann1_berathspm                      float64
ann2_berathspm                      float64
dtype: object

In [28]:
for index, row in df_breaths.iterrows():
    try:
        # set value of breaths per min for the physiological signals data
        df_signal_numeric.loc[
            np.r_[int(row['ann1_prev_breath']):int(row['breaths ann1 [signal sample no]'])], 
            'ann1_breathspm'
        ] = int(row['ann1_berathspm'])
    except Exception as e:
        print(e)
        pass

cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float infinity to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float Na

In [35]:
# save the data to disk
df_signal_numeric.to_csv('./data/merged_v2.csv', index=False)